# Dask Run

In [1]:
import os
import dask.dataframe as dd
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [2]:
# PATH_DATA =  '../../data-testing/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output/'

SLA_NAME = 'dask-sla.pkl'
MEM_NAME = 'dask-mem.pkl'


In [3]:
all_sla = {}
all_memory = {}

In [4]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

In [5]:
# all_sla = joblib.load(PATH_OUTPUT+SLA_NAME)
# all_memory = joblib.load(PATH_OUTPUT+MEM_NAME)

##     Task1 & Task2- I/O

In [6]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [21]:
for i in range(2):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1 = dd.read_csv(PATH_DATA+fn).compute()
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1.to_csv(TEMP_PATH + 'temp.csv',index=False)
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = dd.read_parquet(PATH_DATA+fn, engine='pyarrow').compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.to_parquet(TEMP_PATH + 'temp.parquet',index=False)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(3)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1


In [22]:
all_sla

{'task1': {'csv': {'0': 0.1939930999999433, '1': 1.243986599999971},
  'prq': {'0': 0.1254777000000331, '1': 0.6561456999999109}},
 'task2': {'csv': {'0': 0.47144169999990027, '1': 3.716619600000058},
  'prq': {'0': 0.10739310000008118, '1': 0.8336434000000281}}}

In [23]:
all_memory

{'task1': {'csv': {'0': 0.01705164799999892, '1': 0.13399244800000076},
  'prq': {'0': 0.052219904000001094, '1': 0.11066572800000074}},
 'task2': {'csv': {'0': -0.006619135999999415, '1': -0.029499392000001734},
  'prq': {'0': 0.011849728000001392, '1': 0.0700702720000006}}}

In [24]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [68]:
# all_df = [pd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [4,5,6,7,8,9]]
all_df = [dd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [1,2]]
df_right = dd.read_parquet(PATH_DATA+'data_to_join.parquet')

In [69]:
len(all_df)

2

In [70]:
# type(df_right)

## Task3 - Sorting

In [71]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [72]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.sort_values(['Date','Amount'],ascending=[True,False]).compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


1.03
5.03


In [73]:
all_sla[n_task]

{'1': 0.8413864999999987, '5': 3.559779599999956}

In [74]:
all_memory[n_task]

{'1': 0.040620032000001416, '5': 0.24727142399999913}

## Task4 - Filtering

In [75]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [76]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))].compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


1.03
5.03


In [77]:
all_sla[n_task]

{'1': 0.6573950000001787, '5': 3.0992013000000043}

In [78]:
all_memory[n_task]

{'1': 0.022298624000001155, '5': 0.1374412800000009}

## Task5 - Merging

In [79]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [80]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dd.merge(dtemp1,df_right,on=['Date','ship-service-level'],how='left').compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03


In [81]:
all_sla[n_task]

{'1': 0.787870600000133, '5': 3.537867700000106}

In [82]:
all_memory[n_task]

{'1': 0.051494911999999005, '5': 0.223543295999999}

## Task6 - udf apply

In [83]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [84]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [87]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp1['Amount2'] = dtemp1['Amount'].apply(fun).compute()
    dtemp1 = dtemp1.assign(Amount2 = dtemp1['Amount'].apply(fun, meta=('Amount', 'float64'))).compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03


In [88]:
all_sla[n_task]

{'1': 5.237313300000096, '5': 25.776003100000025}

In [89]:
all_memory[n_task]

{'1': 0.06866943999999897, '5': 0.392974336}

## Task7 - aggregation

In [90]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [91]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [93]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':'mean'}).compute()

    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03


In [94]:
all_sla[n_task]

{'1': 0.3464979999998832, '5': 1.5811969999999747}

In [95]:
all_memory[n_task]

{'1': -0.0007249919999985366, '5': -0.009883648000000633}

## Save results

In [96]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/dask-mem.pkl']